In [1]:
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

In [2]:
import mysql.connector

def connect_to_database():
# Establish a connection to the MySQL database
    conn = mysql.connector.connect(
        host="localhost",
        user="mysql-user",
        password="rootroot",
        database="plantmanager"
    )
    return conn

In [3]:
load_dotenv()

True

In [4]:
endpoint = os.environ.get('ENDPOINT_URL')
deployment = os.environ.get('DEPLOYMENT_NAME')
api_key = os.environ.get('AZURE_OPENAI_KEY')

In [5]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-07-01-preview"
)

In [6]:
def call_openai_function_calling(messages, tools):
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools = tools,
        tool_choice="auto",
        
    )
    return response

In [19]:
def test_database_connection():
    conn = connect_to_database()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT DATABASE();")
        db_name = cursor.fetchone()
        print(f"Connected to: {db_name}")
        cursor.close()
        conn.close()
    else:
        print("Failed to connect to the database.")

In [20]:
print(test_database_connection)

<function test_database_connection at 0x775d8adedab0>


In [33]:
# Define the mapping for metrics and queries
metrics_map = {
    'quantity': f"""
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            SUM(ir.quantity) AS total_quantity_sold
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE( {organization_id}, "-", ""))
        {time_filter}
        GROUP BY product_name
        ORDER BY total_quantity_sold DESC
        LIMIT {limit};
    """,

    'revenue': f"""
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            SUM(ir.quantity * ir.cost_per_unit) AS total_revenue
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE( {organization_id}, "-", ""))
        {time_filter}
        GROUP BY product_name
        ORDER BY total_revenue DESC
        LIMIT {limit};
    """,

    'profit': f"""
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            SUM(ir.quantity * (ir.cost_per_unit - v.purchase_price)) AS total_profit
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE( {organization_id}, "-", ""))
        {time_filter}
        GROUP BY product_name
        ORDER BY total_profit DESC
        LIMIT {limit};
    """,

    'profit_margin': f"""
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            SUM(ir.quantity * (ir.cost_per_unit - v.purchase_price)) AS total_profit,
            SUM(ir.quantity * ir.cost_per_unit) AS total_revenue,
            (SUM(ir.quantity * (ir.cost_per_unit - v.purchase_price)) / SUM(ir.quantity * ir.cost_per_unit)) * 100 AS profit_margin_percentage
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE( {organization_id}, "-", ""))
        {time_filter}
        GROUP BY product_name
        HAVING total_revenue > 0
        ORDER BY profit_margin_percentage DESC
        LIMIT {limit};
    """
}

# Define the mapping for time filters
time_filters = {
    'monthly': "AND YEAR(i.invoice_date) = :year AND MONTH(i.invoice_date) = :month",
    'quarterly': "AND YEAR(i.invoice_date) = :year AND QUARTER(i.invoice_date) = :quarter",
    'half_yearly': "AND YEAR(i.invoice_date) = :year AND CASE WHEN MONTH(i.invoice_date) BETWEEN 1 AND 6 THEN 1 ELSE 2 END = :half_year",
    'annually': "AND YEAR(i.invoice_date) = :year"
}

def get_top_products_by_metric_and_time(metric, time_period, organization_id, limit=10, **time_params):
    # Select the correct metric query and time filter
    query = metrics_map.get(metric).format(
        time_filter=time_filters.get(time_period),
        limit=limit
    )

    # Add time filter parameters like year, month, etc.
    # Execute the query with the provided time_params

    # Example of execution
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute(query, {
        'organization_id': organization_id,
        **time_params
    })
    result = cursor.fetchall()
    return result


NameError: name 'time_filter' is not defined

In [9]:
def get_top_products(limit, metric, time_period, specific_period, organization_id, start_date, end_date):
    conn = connect_to_database()  # Establish the database connection
    cursor = conn.cursor()

    # Common SQL for filtering by time period
    time_period_sales = f"""
    WITH time_period_sales AS (
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            ir.quantity * (ir.cost_per_unit - v.purchase_price) AS profit,
            ir.quantity AS quantity_sold,
            i.invoice_date,
            YEAR(i.invoice_date) AS year,
            MONTH(i.invoice_date) AS month,
            QUARTER(i.invoice_date) AS quarter,
            CASE 
                WHEN MONTH(i.invoice_date) BETWEEN 1 AND 6 THEN 1
                ELSE 2
            END AS half_year
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE(%s, "-", ""))
            AND i.invoice_date BETWEEN %s AND %s
    )
    """

    # Build the query based on the metric
    if metric == "profit":
        query = f"""
        {time_period_sales}
        SELECT 
            product_name,
            SUM(profit) AS total_profit,
            SUM(quantity_sold) AS total_quantity_sold,
            %s AS time_period
        FROM time_period_sales
        WHERE CASE 
            WHEN %s = 'monthly' THEN CONCAT(year, '-', LPAD(month, 2, '0'))
            WHEN %s = 'quarterly' THEN CONCAT(year, '-Q', quarter)
            WHEN %s = 'half_yearly' THEN CONCAT(year, '-H', half_year)
            ELSE 'all'
        END = %s
        GROUP BY product_name
        ORDER BY total_profit DESC
        LIMIT %s;
        """
    elif metric == "profit_margin":
        query = """
        SELECT 
            CASE 
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            SUM(ir.quantity * (ir.cost_per_unit - v.purchase_price)) AS total_profit,
            SUM(ir.quantity * ir.cost_per_unit) AS total_revenue,
            (SUM(ir.quantity * (ir.cost_per_unit - v.purchase_price)) / SUM(ir.quantity * ir.cost_per_unit)) * 100 AS profit_margin_percentage,
            SUM(ir.quantity) AS total_quantity_sold
        FROM invoice_row ir
        JOIN invoice i ON ir.invoice = i.id
        JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE(%s, "-", ""))
        GROUP BY product_name
        HAVING total_revenue > 0
        ORDER BY profit_margin_percentage DESC
        LIMIT %s;
        """
    elif metric == "quantity":
        query = f"""
        {time_period_sales}
        SELECT 
            product_name,
            SUM(quantity_sold) AS total_quantity_sold,
            %s AS time_period
        FROM time_period_sales
        WHERE CASE 
            WHEN %s = 'monthly' THEN CONCAT(year, '-', LPAD(month, 2, '0'))
            WHEN %s = 'quarterly' THEN CONCAT(year, '-Q', quarter)
            WHEN %s = 'half_yearly' THEN CONCAT(year, '-H', half_year)
            ELSE 'all'
        END = %s
        GROUP BY product_name
        ORDER BY total_quantity_sold DESC
        LIMIT %s;
        """
    else:
        query = None

    # Execute the query with provided parameters
    if query:
        cursor.execute(query, (organization_id, start_date, end_date, time_period, time_period, time_period, time_period, specific_period, limit))
        result = cursor.fetchall()
    else:
        result = []

    # Close the connection
    cursor.close()
    conn.close()

    return result

In [ ]:
organization_id = '7f366021-c66a-4fdc-99ae-bdfce2113cb2'  # Replace with your organization ID
start_date = '2023-01-01'
end_date = '2023-12-31'
time_period = 'monthly'
specific_period = '2023-06'

result_quantity_monthly = get_top_products(
    metric='quantity',
    organization_id=organization_id,
    limit=1,
    start_date=start_date,
    end_date=end_date,
    specific_period=specific_period,
    time_period='monthly'
    
)
print("Top products by quantity (monthly):", result_quantity_monthly)


In [32]:
organization_id = '7f366021-c66a-4fdc-99ae-bdfce2113cb2'
year = 2023
month = 8
limit = 5

result_revenue_quarterly = get_top_products_by_metric_and_time(
    metric='revenue',
    time_period='quarterly',
    organization_id=organization_id,
    limit=limit,
    year=year,
    quarter=2  # Second quarter (April - June)
)
print("Top products by revenue (quarterly):", result_revenue_quarterly)

KeyError: 'organization_id'

In [ ]:
SET @organization_id = '123e4567-e89b-12d3-a456-426614174000';
SET @start_date = '2023-01-01';
SET @end_date = '2023-12-31';
SET @time_period = 'monthly';
SET @specific_period = '2023-06';

-- Then run the main query with these variables

In [ ]:
def get_top_products(conn, limit, metric='quantity', time_period='monthly'):
    # Mapping metric types to SQL expressions
    metrics_map = {
        'quantity': 'SUM(ir.quantity)',
        'revenue': 'SUM(ir.quantity * ir.price)',
        'profit': 'SUM((ir.price - ir.cost_price) * ir.quantity)',
        'profit_margin': 'SUM((ir.price - ir.cost_price) * ir.quantity) / SUM(ir.price * ir.quantity)'
    }
    
    # Time period SQL filtering
    time_filters = {
        'monthly': 'AND i.order_date >= DATE_SUB(NOW(), INTERVAL 1 MONTH)',
        'quarterly': 'AND i.order_date >= DATE_SUB(NOW(), INTERVAL 3 MONTH)',
        'half_yearly': 'AND i.order_date >= DATE_SUB(NOW(), INTERVAL 6 MONTH)',
        'annually': 'AND i.order_date >= DATE_SUB(NOW(), INTERVAL 1 YEAR)'
    }

    metric_expression = metrics_map.get(metric, 'SUM(ir.quantity)')
    time_filter = time_filters.get(time_period, '')

    # SQL query
    query = f"""
        SELECT
            CASE
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END AS product_name,
            {metric_expression} AS metric_value
        FROM sales_order_row ir
        JOIN sales_order i ON ir.sales_order = i.id
        LEFT JOIN variant v ON ir.variant = v.id
        LEFT JOIN product p ON v.product_id = p.id
        LEFT JOIN material m ON v.material_id = m.id
        WHERE i.organization = UNHEX(REPLACE("7f366021-c66a-4fdc-99ae-bdfce2113cb2", "-", ""))
        {time_filter}
        GROUP BY
            CASE
                WHEN p.id IS NOT NULL THEN p.name
                ELSE m.name
            END
        ORDER BY metric_value DESC
        LIMIT {limit};
    """

    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return result
